In [1]:
import pandas as pd


In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
df = pd.read_csv('../data/data.csv')

In [4]:
documents = df.to_dict(orient='records')

In [5]:
documents

[{'id': 0,
  'exercise_name': 'Push-Ups',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Upper Body',
  'type': 'Push',
  'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
  'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'},
 {'id': 1,
  'exercise_name': 'Squats',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Push',
  'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings',
  'instructions': 'Stand with feet shoulder-width apart. Lower your body as if sitting back into a chair, keeping your chest up. Return to standing.'},
 {'id': 2,
  'exercise_name': 'Plank',
  'type_of_activity': 'Strength/Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Core',
  'type': 'Hold',
  'muscle_groups_activated': 'Rectus Abdominis, Transvers

In [6]:
prompt_template = """
You emulate a user of our fitness assistance application.
Formulate 5 questions this user might ask based on a provided exercise. 
The record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

'exercise_name': {exercise_name}
'type_of_activity': {type_of_activity}
'type_of_equipment': {type_of_equipment}
'body_part': {body_part}
'type': {type}
'muscle_groups_activated': {muscle_groups_activated}
'instructions': {instructions}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [7]:
print(prompt_template.format(**documents[0]))

You emulate a user of our fitness assistance application.
Formulate 5 questions this user might ask based on a provided exercise. 
The record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

'exercise_name': Push-Ups
'type_of_activity': Strength
'type_of_equipment': Bodyweight
'body_part': Upper Body
'type': Push
'muscle_groups_activated': Pectorals, Triceps, Deltoids
'instructions': Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]


In [8]:
prompt = prompt_template.format(**documents[0])


In [9]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o-mini', 
        messages = [{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content

In [10]:
questions = llm(prompt)

In [11]:
import json

In [12]:
json.loads(questions)

{'questions': ['What muscles are targeted when performing push-ups?',
  'What is the starting position for doing push-ups?',
  'Do I need any equipment to perform push-ups?',
  'What should I do after lowering my body during a push-up?',
  'Which body part is primarily worked out by push-ups?']}

In [13]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [14]:
from tqdm.auto import tqdm


In [15]:
results = {}


In [62]:
results = list(results.items())

In [68]:
results = results[19:]

# Convert the sliced list back to a dictionary
results = dict(results)


TypeError: unhashable type: 'slice'

In [69]:
type(results)

dict

In [67]:
final_results = []

for doc_id, info in results.items():
    # Ensure 'info' is a dictionary and contains 'questions'
    if isinstance(info, dict):
        questions = info.get('questions', [])
        # Iterate over each question
        for question in questions:
            # Append a dictionary for each question
            final_results.append({'doc_id': doc_id, 'question': question})
    else:
        print(f"Unexpected type for doc_id {doc_id}: {type(info)}")

Unexpected type for doc_id 55: <class 'list'>
Unexpected type for doc_id 158: <class 'list'>


In [ ]:
final_results[0]

In [ ]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])

In [ ]:
df_results.to_csv('../data/ground-truth-retrieval.csv', index=False)